# SCP Toolbox Workshop

___A tutorial on generating dynamically feasible trajectories reliably and efficiently___

Monday, February 7, 2022

Rocky Mountain AAS GN&C Conference, Breckenridge, CO

# Part 1

In [1]:
import Pkg
Pkg.activate("..")

# these lines are required only for local installations
Pkg.develop(path="../../scp_traj_opt/")
Pkg.precompile()

using SCPToolbox
using ECOS
using OSQP
using SCS
using Ipopt

  Activating project at `~/Documents/SCPToolbox_tutorial`


## Conic Linear Programs

__General form__:

\begin{align}
\underset{x\in\mathbb{R}^n}{{\text{minimize}}} &~ c^\top x \\
\text{subject to} &~Ax+b \in \mathbb{K}
\end{align}

where $c\in\mathbb{R}^n,~b\in\mathbb{R}^m,~A\in\mathbb{R}^{m\times n}$ are problem parameters, and $\mathbb{K}$ is a cartesian product of convex cones. The cones supported by `SCPToolbox.jl` are described in the table below.

The optimization problem shown here can be solved using state-of-the-art solvers like `ECOS`, `Gurobi` and `Mosek`. However, they typically need a parser to convert the problem data to a canonical form. Hence such solvers are not user-friendly to interface with and solve problems which are not natively expressed in the canonical forms. 

In `SCPToolbox.jl`, henceforth refered to as the toolbox, a conic linear program is formulated by instantiating a `ConicProblem` object with the choice of solver and its associated options.  

|    Label        | Notation     |     Name      | Definition |
|:---------------:|:----:|:-------------:|:----------:|
|`UNCONSTRAINED`  |  $\mathbb{K}_{ \lesseqgtr}$ | Unconstrained cone  |  $$\{x\in\mathbb{R}^n\}$$ |
|`ZERO`           |  $\mathbb{K}_{=0}$ | Zero cone             | $\{x\in\mathbb{R}^n\mid x=0\}$                               |
|`NONPOS`         |  $\mathbb{K}_{\leq 0}$ | Non-positive orthant  | $\{x\in\mathbb{R}^n\mid x\leq 0\}$                           |
|`L1`             |  $\mathbb{K}_{1}$ | $l_1$ norm cone       | $\{(t,x)\in\mathbb{R}^n\times\mathbb{R}\mid \|x\|_1\leq t\}$ |
|`SOC`             |  $\mathbb{K}_{2}$ | $l_2$ norm cone       | $\{(t,x)\in\mathbb{R}^n\times\mathbb{R}\mid \|x\|_2\leq t\}$ |
| `LINF` |  $\mathbb{K}_{\infty}$ |  $l_\infty$ norm cone |  $\{(t,x)\in\mathbb{R}^n\times\mathbb{R}\mid \|x\|_\infty\leq t\}$  |
| `GEOM` |  $\mathbb{K}_{\text{GM}}$ | Geometric mean cone | $ \big\{ (t,x)\in\mathbb{R}^n\times\mathbb{R}\mid \left(\prod_{i=1}^n x_i\right)^{1/n} \geq t \big\}$ |
| `EXP` |  $\mathbb{K}_{\text{EXP}}$ | Exponential cone | $\{(x,y,z)\in \mathbb{R}\times \mathbb{R} \times \mathbb{R} \mid y\,\text{exp}(x/y) \leq z,~y>0\}$ |

In [2]:
opts = Dict("verbose" => 0)
pbm = ConicProgram(solver = ECOS.Optimizer, solver_options = opts)

Conic linear program

  Feasibility problem
  0 variables (0 blocks)
  0 parameters (0 blocks)
  0 constraints

  Variable argument
    0 elements
    0 blocks


  Parameter argument
    0 elements
    0 blocks


## Variables

Variables are the quantities that the SCP Toolbox determines values for upon solution. These variables therefore begin as symbolic values during problem formulation, but are ultimately given numerical values upon convergence to a locally optimal trajectory solution. These variables are generally broken up into three main categories: 

- **scalars**: variables containing a single element.  *e.g.* $x \in \mathbb{R}$.
- **vectors**: variables containing a single row or column.  *e.g.* $x \in \mathbb{R}^N$.
- **matrices**: variables for both rows and columns greater than size two.  *e.g.* $x \in \mathbb{R}^{N \times M}$.

### Variable Properties

Each variable declared by the user is defined by a set of properties.

1. **Elements:** The number of elements in the variable, given as a scalar.
2. **Shape:** The tuple defining the dimensions of the variable, given as $(N,M)$ where $N$ is the number of rows and $M$ is the number of columns. This appears as $(1,)$ for a scalar, $(N,)$ for an $N$-dimensional vector, and $(N,M)$ for an $N \times M$-dimensional matrix. 
3. **Name:** The name is a colloquial title given by the user. This property not only allows the user to keep track of the origin of the variables of the problem, but can also be queried to search for all variables with names containing a common string.
4. **Block index:** The block index keeps track of the order in which each standalone variable is stored in the stack. Though the user defines each variable individually, the parser reformats these variables into a single vector, where the data within each element of the variable is stored in a corresponding block of elements. Vectors and scalars are simply stored in the stack same format, while matrices are vectorized in column-major form.
5. **Indices:** The indices of the variable define the explicit elements of the stack where the variable data is stored. The elements in these indices make up the block mentioned above.
6. **Type:** The category of the variable.
7. **Scaling Type:** The information on whether a variable is scaled, and how scaling is performed if so. Scaling means that the physical quantity for a variable is nondimensionalized with an affine mapping when exposed to the optimizer. This scaling is reversed upon achieving a solution to achieve a meaningful physical quantity.
8. **Value Type:** The symbolic or numerical quantity assigned to a given variable.

### Variable Declaration

At the high level, each variable declaration is done with the `@new_variable()` function. However, this constructor is overloaded such that variables may be declared with a variety of different input-argument syntaxes. Each call to the constructor takes in the corresponding problem that they are associated with alongside other input arguments.

A scalar variable may be declared via passing the corresponding problem, a dimension of 1, and a variable name as constructor inputs, shown respectively below. The resulting variable properties are shown as well. Here, scaling is not applied.

In [3]:
x = @new_variable(pbm, 1, "x")

Vector variable
  1 elements
  (1,) shape
  Name: x
  Block index in stack: 1
  Indices in stack: 1
  Type: Vector{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     x

An equivalent scalar variable may also be defined as follows, where the input argument of the variable dimensions are modified. Note that for each new variable definition within a given problem object, the block index increments by 1 and the corresponding stack indices also increase according to the next available element in the stack and the size of the new variable.

In [4]:
x = @new_variable(pbm, (1,1), "x")

Matrix variable
  1 elements
  (1, 1) shape
  Name: x1
  Block index in stack: 2
  Indices in stack: 2
  Type: Matrix{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     x1

Finally, a scalar quantity may be simply defined as follows, without specifying a dimension.

In [5]:
q = @new_variable(pbm, "q")

Vector variable
  1 elements
  (1,) shape
  Name: q
  Block index in stack: 3
  Indices in stack: 3
  Type: Vector{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     q


A vector variable is declared by defining a length dimension $>1$. Note the value now has multiple elements.

In [6]:
v = @new_variable(pbm, 3, "v")

Vector variable
  3 elements
  (3,) shape
  Name: v
  Block index in stack: 4
  Indices in stack: 4:6
  Type: Vector{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     v[1]
     v[2]
     v[3]

A matrix variable may be declared by defining dimensions $>1$ for both rows and columns. Note that the value property now has indices that mirror the shape property. A matrix variable may represent a standard matrix, such as those commonly applied to achieve a linear transformations on a vector. A matrix variable may also represent a set of vectors, such as values of the state or control from a dynamic system, sampled at variable discrete timesteps along a trajectory.

In [7]:
y = @new_variable(pbm, (2, 5), "y")

Matrix variable
  10 elements
  (2, 5) shape
  Name: y
  Block index in stack: 5
  Indices in stack: 7:16
  Type: Matrix{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     y[1,1]  y[1,2]  y[1,3]  y[1,4]  y[1,5]
     y[2,1]  y[2,2]  y[2,3]  y[2,4]  y[2,5]

Individual row or column vectors of a matrix may be indexed as follows. Note that these indexed quantities are themselves vector variables.

In [8]:
y[:, 1]

Vector variable
  2 elements
  (2,) shape
  Name: y
  Block index in stack: 5
  Indices in stack: 7:8
  Type: SubArray{JuMP.AffExpr, 1, Matrix{JuMP.AffExpr}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64}, true}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     y[1,1]
     y[2,1]

### Variable Operations



Once a variable is defined, additional operations may be performed to scale the variable and probe its value. Scaling is an important operation which improves numerical conditioning of the underlying optimization problem by applying an affine mapping to the physical quantity in order to produce a corresponding scaled quantity that lies approximately between 0 and 1. In terms a notation, a physical quantity may be denoted as $x$ while the corresponding scaled quantity may be denoted as $\hat{x}$.

For a scalar, the affine mapping may be approximated as follows. $x = a*\hat{x} + b$, where $a = (x_{max} - x_{min})$ and $b = -x_{min}$.

The utility of the affine mapping becomes clear while defining simple set constraints on the value of the variable. Examples of such constraints are a ball, halfspace, or box, etc. This is discussed in more detail in the Constraints section below.

The affine mapping for scaling a given physical variable is called via the `@scale()` function. This function is given the vector to be scaled, a vector of entries that will linearly scale each element of the variable, and an affine offset, respectively.

In [9]:
qh = @new_variable(pbm, "qh")

Vector variable
  1 elements
  (1,) shape
  Name: qh
  Block index in stack: 6
  Indices in stack: 17
  Type: Vector{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     qh

In [10]:
@scale(qh, [10], 5)

The value of this new affine mapping can be queried with the `value()` function as follows.

In [11]:
value(qh)

1-element Vector{JuMP.AffExpr}:
 10 qh + 5

To apply a different scaling to each element of a variable, diagonal entires of a matrix that scales each variable element may be defined. In addition, vector-value affine offsets may also be applied. 

In [12]:
@scale(y, [2; 4], [3;1])

In [13]:
y

Matrix variable
  10 elements
  (2, 5) shape
  Name: y
  Block index in stack: 5
  Indices in stack: 7:16
  Type: Matrix{JuMP.AffExpr}
  Affine scaling x=(S.*xh).+c
  Any perturbation allowed
  Value =
     y[1,1]  y[1,2]  y[1,3]  y[1,4]  y[1,5]
     y[2,1]  y[2,2]  y[2,3]  y[2,4]  y[2,5]

The value of this scaled matrix variable is given below.

In [14]:
value(y)

2×5 Matrix{JuMP.AffExpr}:
 2 y[1,1] + 3  2 y[1,2] + 3  2 y[1,3] + 3  2 y[1,4] + 3  2 y[1,5] + 3
 4 y[2,1] + 1  4 y[2,2] + 1  4 y[2,3] + 1  4 y[2,4] + 1  4 y[2,5] + 1

After defining the variables above, the properties of the corresponding optimization problem may be queried.

In [15]:
pbm

Conic linear program

  Feasibility problem
  17 variables (6 blocks)
  0 parameters (0 blocks)
  0 constraints

  Variable argument
    17 elements
    6 blocks
     1) 1    ... x
     2) 2    ... x1
     3) 3    ... q
     4) 4:6  ... v
     5) 7:16 ... y
     6) 17   ... qh

  Parameter argument
    0 elements
    0 blocks


Finally, the variables of this problem can be queried explicitly.

In [16]:
variables(pbm)

6-element Vector{SCPToolbox.Parser.ConicLinearProgram.VariableArgumentBlock}:
 Vector variable
  1 elements
  (1,) shape
  Name: x
  Block index in stack: 1
  Indices in stack: 1
  Type: Vector{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     x
 Matrix variable
  1 elements
  (1, 1) shape
  Name: x1
  Block index in stack: 2
  Indices in stack: 2
  Type: Matrix{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     x1
 Vector variable
  1 elements
  (1,) shape
  Name: q
  Block index in stack: 3
  Indices in stack: 3
  Type: Vector{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     q
 Vector variable
  3 elements
  (3,) shape
  Name: v
  Block index in stack: 4
  Indices in stack: 4:6
  Type: Vector{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     v[1]
     v[2]
     v[3]
 Matrix variable
  10 elements
  (2, 5) shape
  Name: y
  Block index in stack: 5
  Indices in stack: 7:16
  Type: Ma

In [17]:
display(variables(pbm))

6-element Vector{SCPToolbox.Parser.ConicLinearProgram.VariableArgumentBlock}:
 Vector variable
  1 elements
  (1,) shape
  Name: x
  Block index in stack: 1
  Indices in stack: 1
  Type: Vector{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     x
 Matrix variable
  1 elements
  (1, 1) shape
  Name: x1
  Block index in stack: 2
  Indices in stack: 2
  Type: Matrix{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     x1
 Vector variable
  1 elements
  (1,) shape
  Name: q
  Block index in stack: 3
  Indices in stack: 3
  Type: Vector{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     q
 Vector variable
  3 elements
  (3,) shape
  Name: v
  Block index in stack: 4
  Indices in stack: 4:6
  Type: Vector{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     v[1]
     v[2]
     v[3]
 Matrix variable
  10 elements
  (2, 5) shape
  Name: y
  Block index in stack: 5
  Indices in stack: 7:16
  Type: Ma

## Constraints

A conic constraint $y\in\mathbb{K}$ (say for $\mathbb{K} = \mathbb{K}_2$) can be modelled in the toolbox as follows:

In [18]:
params = Dict("n" => 4, "a" => 100)
opts = Dict("verbose" => 0)
pbm = ConicProgram(params; solver = ECOS.Optimizer, solver_options = opts)

x = @new_variable(pbm, length(params["n"]),"x")
t = @new_variable(pbm,"t")

# Define the constraint
cstr = @add_constraint(
    pbm, SOC, "my-soc", (x, t),
    begin
        local x, t = arg
        vcat(pars["a"]*t, x)
    end)

Name: my-soc
Cone f(x,p)∈K, where:
K is a second-order cone, {(t, x)∈ℝ×ℝⁿ : ‖x‖₂≤t}
f(x,p) = 
  100 t
  x
Affine function
Arguments:
  x (block 1) : 1
  t (block 2) : 2


In [19]:
variables(pbm,"^x\$") == x

true

Note the use of [regular expressions](https://en.wikipedia.org/wiki/Regular_expression) for searching though the list of variables names.

In [20]:
constraints(pbm, "^my-soc")

Name: my-soc
Cone f(x,p)∈K, where:
K is a second-order cone, {(t, x)∈ℝ×ℝⁿ : ‖x‖₂≤t}
f(x,p) = 
  100 t
  x
Affine function
Arguments:
  x (block 1) : 1
  t (block 2) : 2


## Examples

Many simple constraint set typical in a trajectory optimization problem like box, ball, halfspace and hyperplane can be represented as conic constraints.

### Box

A box with bounds $l,u\in\mathbb{R}$ with $l<u$ can be formulated as as non-positive orthant cone:

$$
\{x\in\mathbb{R}^n~|~l\leq x \leq u\}
$$

$$
l\leq x \leq u \iff \left[ \begin{array}{c}I\\ -I\end{array}\right]x + \left[\begin{array}{c}-u\mathbf{1}\\ l\mathbf{1} \end{array}\right] \in \mathbb{K}_{\leq 0}
$$

In [21]:
params = Dict("n" => 2, "l" => 2, "u" => 9)
A = [1  0;0 1;-1 0;0 -1]
b = [-params["u"];-params["u"];params["l"];params["l"]]

pbm = ConicProgram(params)

x = @new_variable(pbm, params["n"],"x")

# Define the constraint
cstr = @add_constraint(
    pbm, NONPOS, "my-box", (x,),
    begin
        local x, = arg
        A*x+b
    end)

Name: my-box
Cone f(x,p)∈K, where:
K is a nonpositive orthant cone, {z : z≤0}
f(x,p) = 
  x[1] - 9
  x[2] - 9
  -x[1] + 2
  -x[2] + 2
Affine function
Arguments:
  x (block 1) : 1:2


### Ball

A ball with radius $r \in \mathbb{R}_+$ set can be represented as a $l_2$ norm cone (second-order cone):

$$
\{x\in\mathbb{R}^n~|~\|x\|_2\leq r\}
$$

$$
\|x\|_2\leq r \iff \left[ \begin{array}{c} 0 \\ I \end{array} \right] x + \left[ \begin{array}{c} r \\ 0 \end{array} \right]\in \mathbb{K}_2
$$

In [22]:
params = Dict("n" => 2, "r" => 4)
A = [0  0;1 0;0 1]
b = [params["r"];0;0]

pbm = ConicProgram(params)

x = @new_variable(pbm, params["n"],"x")

# Define the constraint
cstr = @add_constraint(
    pbm, SOC, "my-ball", (x,),
    begin
        local x, = arg
        A*x+b
    end)

Name: my-ball
Cone f(x,p)∈K, where:
K is a second-order cone, {(t, x)∈ℝ×ℝⁿ : ‖x‖₂≤t}
f(x,p) = 
  4
  x[1]
  x[2]
Affine function
Arguments:
  x (block 1) : 1:2


### Hyperplane

A hyperplane defined by $a\in \mathbb{R}^n$ and $b\in\mathbb{R}$ can be represented as a zero cone:

$$
\left\{x\in\mathbb{R}^n~|~a^\top x = b\right\}
$$

$$
a^\top x = b \iff \left[a^\top\right]x + (-b) \in \mathbb{K}_{=0}
$$

In [23]:
params = Dict("n" => 2, "a" => [1.1;3.2], "b" => 9)
A = Matrix(transpose(params["a"]))
b = -params["b"]

@show A

pbm = ConicProgram(params)

x = @new_variable(pbm, params["n"],"x")

# Define the constraint
cstr = @add_constraint(
    pbm, ZERO, "my-hypln", (x,),
    begin
        local x, = arg
        A*x .+ b
    end)

A = [1.1 3.2]


Name: my-hypln
Cone f(x,p)∈K, where:
K is a zero cone, {z : z=0}
f(x,p) = 
  1.1 x[1] + 3.2 x[2] - 9
Affine function
Arguments:
  x (block 1) : 1:2


### Halfspace

A halfspace defined by $a\in \mathbb{R}^n$ and $b\in\mathbb{R}$ can be represented as a non-positive orthant cone:

$$
\left\{x\in\mathbb{R}^n~|~a^\top x \leq b\right\}
$$

$$
a^\top x \leq b \iff \left[a^\top\right] x + (-b) \in \mathbb{K}_{\leq 0}
$$

In [24]:
params = Dict("n" => 2, "a" => [1.1;3.2], "b" => 9)
A = Matrix(transpose(params["a"]))
b = -params["b"]

@show A*[1;2]

pbm = ConicProgram(params)

x = @new_variable(pbm, params["n"],"x")

# Define the constraint
cstr = @add_constraint(
    pbm, NONPOS, "my-hlfspace", (x,),
    begin
        local x, = arg
        println(size(A*x))
        A*x .+ b
    end)

A * [1; 2] = [7.5]
(1,)


Name: my-hlfspace
Cone f(x,p)∈K, where:
K is a nonpositive orthant cone, {z : z≤0}
f(x,p) = 
  1.1 x[1] + 3.2 x[2] - 9
Affine function
Arguments:
  x (block 1) : 1:2


## Objective function

 - This section explains how the __objective function__ is defined. 

 - Regardless of whether optimization problems have constraints or not, they seek to optimize a function with respect to  a chosen _objective_.

 - i.e., they seek to either minimize a certain _cost_ or maximize a certain _reward_. 

 - For example, propellant consumption is an important metric that is ideally optimized for (minimized) in a rocket-powered planetary landing mission. We may also want to minimize the total energy to go from one point to another.

__Note__: Although we take cost-minimization to be the default objective here, a problem can be easily cast as a reward-maximization problem by minimizing the negative of the objective function.

Let us define a new optimization problem, but this time, with a user-defined constant parameter, `a`.

In [25]:
my_pars = Dict("a" => 5)
opts = Dict("verbose" => 0)
pbm = ConicProgram(my_pars; solver = ECOS.Optimizer, solver_options = opts)

Conic linear program

  Feasibility problem
  0 variables (0 blocks)
  0 parameters (0 blocks)
  0 constraints

  Variable argument
    0 elements
    0 blocks


  Parameter argument
    0 elements
    0 blocks


Next, we define a scalar variable, `x`.

In [26]:
x = @new_variable(pbm, "x")

Vector variable
  1 elements
  (1,) shape
  Name: x
  Block index in stack: 1
  Indices in stack: 1
  Type: Vector{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     x

Now we are ready to define our objective function, which we will henceforth refer to as simply "cost".

By default, the cost is initialized to zero when a new problem is generated. We use the `@add_cost` function to add terms to the cost `obj`.

In [27]:
obj = @add_cost(
                pbm, (x,), 
                begin
                    x, = arg
                    pars["a"]*x
                end
                )

Cost function composed of 1 terms

Term 1:
  Coefficient: 1.00e+00
  Affine function
  Arguments:
    x (block 1) : 1
  Current value =
    5 x

 - The first argument of the function is to indicate that we are adding the cost term to the problem we have defined, `pbm`. 

 - The third argument unpacks the tuple into variables that are used to define the cost. Here, our cost is simply the constant parameter that we defined earlier, `a`, and the cost itself is `a*x`.

 - The second argument is a tuple including the variables associated with the cost, which in this case is just `x`.

The cost function we have included is as follows:

In [28]:
cost(pbm)

Cost function composed of 1 terms

Term 1:
  Coefficient: 1.00e+00
  Affine function
  Arguments:
    x (block 1) : 1
  Current value =
    5 x

## Solve problem

With everything we have learned so far, we are now ready to solve a convex optimization problem!

We will start with a new problem for this example.

As before, we begin by defining constant parameters to be passed to the solver.

In [29]:
problem_pars = Dict("a" => 5, "x_ref" => [2; 2; 5; 10; -1])

Dict{String, Any} with 2 entries:
  "x_ref" => [2, 2, 5, 10, -1]
  "a"     => 5

Here, we have defined one scalar constant, `a`, and one 5-dimensional constant vector, `x_ref`.

Now, we create a dictionary of options to be passed to the low-level optimizer. In this case, we set the `verbose` flag to `1` so that we can see the full output of the low-level optimizer.

In [30]:
opts = Dict("verbose" => 1)

Dict{String, Int64} with 1 entry:
  "verbose" => 1

We now define our optimization problem. We shall make use of the open-source low-level convex optimization solver, ECOS.

In [31]:
pbm = ConicProgram(problem_pars;
    solver = ECOS.Optimizer,
    solver_options = opts)

Conic linear program

  Feasibility problem
  0 variables (0 blocks)
  0 parameters (0 blocks)
  0 constraints

  Variable argument
    0 elements
    0 blocks


  Parameter argument
    0 elements
    0 blocks


We define two variables, `t` of dimension 1, and `x` of dimension 5.

In [32]:
x = @new_variable(pbm, length(problem_pars["x_ref"]), "x")
t = @new_variable(pbm, "t")

Vector variable
  1 elements
  (1,) shape
  Name: t
  Block index in stack: 2
  Indices in stack: 6
  Type: Vector{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     t

Now we add a second-order cone (SOC) constraint, $\|x-x_{ref}\|_2 \le t$, to the problem.

In [33]:
@add_constraint(
    pbm, SOC, (x, t),
    begin
        x, t = arg
        vcat(t, x-pars["x_ref"])
    end)

Name: f1
Cone f(x,p)∈K, where:
K is a second-order cone, {(t, x)∈ℝ×ℝⁿ : ‖x‖₂≤t}
f(x,p) = 
  t
  x[1] - 2
  x[2] - 2
  x[3] - 5
  x[4] - 10
  x[5] + 1
Affine function
Arguments:
  x (block 1) : 1:5
  t (block 2) : 6


We choose the cost function to be $t^{2}$.

In [34]:
@add_cost(
    pbm, (t,), 
    begin
        t, = arg
        t.^2
    end)

Cost function composed of 1 terms

Term 1:
  Coefficient: 1.00e+00
  Quadratic function
  Arguments:
    t (block 2) : 6
  Current value =
    t²

We are finally ready to solve the problem we just defined!

In [35]:
exit_status = solve!(pbm)


ECOS 2.0.8 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -2.449e-01  +4e+00  2e-01  1e-01  1e+00  2e+00    ---    ---    1  1  - |  -  - 
 1  -6.147e-02  -8.099e-02  +2e-01  7e-03  4e-03  3e-02  9e-02  0.9530  5e-03   1  1  1 |  0  0
 2  -5.548e-04  -2.077e-03  +1e-02  3e-04  2e-04  9e-04  5e-03  0.9475  8e-04   1  1  1 |  0  0
 3  -4.848e-04  -6.068e-04  +2e-03  6e-05  4e-05  4e-04  1e-03  0.9090  1e-01   1  2  2 |  0  0
 4  -4.348e-05  -7.855e-05  +8e-04  2e-05  1e-05  1e-04  4e-04  0.7244  1e-01   1  2  2 |  0  0
 5  -1.174e-04  -1.268e-04  +3e-04  9e-06  5e-06  7e-05  2e-04  0.8239  3e-01   1  1  1 |  0  0
 6  -1.708e-06  -3.017e-06  +5e-05  1e-06  7e-07  9e-06  2e-05  0.9890  1e-01   2  2  2 |  0  0
 7  -6.230e-08  -1.239e-07  +2e-06  6e-08  4e-08  4e-07  1e-06  0.9600  8e-03   1  1  1 |  0  0
 8  -5.661e-08  -6.196e-08  +2e-07  5e-09  3e-

OPTIMAL::TerminationStatusCode = 1

The optimal cost value is `J_opt`.

In [36]:
J_opt = objective_value(pbm)

-8.29959449100464e-10

The optimal values of the decision variables are `x_opt` and `t_opt`.

In [37]:
x_opt = value(x)

5-element Vector{Float64}:
  2.0000000000095266
  2.0000000000095266
  5.0000000000238165
 10.000000000047633
 -1.000000000004763

In [38]:
t_opt = value(t)

1-element Vector{Float64}:
 1.1524313115706931e-5

The optimal values of the decision variables can also be accessed as mentioned before.

In [39]:
value(variables(pbm, "^x\$"))

5-element Vector{Float64}:
  2.0000000000095266
  2.0000000000095266
  5.0000000000238165
 10.000000000047633
 -1.000000000004763

In [40]:
value(variables(pbm, "^t\$"))

1-element Vector{Float64}:
 1.1524313115706931e-5

Congratulations! 

We now know how to solve convex optimization problems, which form the basis for Sequential Convex Programming (SCP) problems, which we shall now get into with some exciting examples!